In [1]:
from network import Net
from trainers import *
from validator import *
import numpy as np
import pandas as pd

In [2]:
#load data TRAIN
data = pd.read_csv(r'C:\Users\Tano\Documents\UNI\MAGISTRALE\ML\nn_from_scratch\datasets\cup\ML-CUP24-TR.csv', header=None, skiprows=7)
data = data.to_numpy()

#split data in train 80% and test 20%
np.random.shuffle(data)
train = data[:int(len(data)*0.8)]
test = data[int(len(data)*0.8):]

#split in x and y
x_train = train[:,1:-3]
y_train = train[:,-3:]
x_test = test[:,1:-3]
y_test = test[:,-3:]

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(200, 12)
(200, 3)
(50, 12)
(50, 3)


In [4]:
network = Net('./config_cup.yaml')
network.print_structure()

Layer 1: Units = 768, Activation = relu
Layer 2: Units = 768, Activation = relu
Layer 3: Units = 3, Activation = identity


In [5]:
# first grid search
valid = Validator('param_grid_cup.yaml')
print(len(valid.grid))

best_configuration = valid.grid_search(x_train, y_train, network)
best_configuration

648


100%|██████████| 648/648 [1:40:58<00:00,  9.35s/it]


{'conf': {'activations': ['relu', 'relu', 'identity'],
  'alpha': 0.9,
  'eta': 0.001,
  'initializer': 'he',
  'lambd': 1e-05,
  'units': [1024, 1024, 3]},
 'epochs': 402,
 'metrics': np.float64(4.077944746740005)}

In [ ]:
#second grid search, fine grained
valid = Validator('param_grid_cup2.yaml')
print(len(valid.grid))

best_configuration = valid.grid_search(x_train, y_train, network)
best_configuration

In [ ]:
network.set_best_configuration(best_configuration)
model = train_model(x_train, y_train, network.get_W(), network.get_b(), network.get_layers(), network.get_hyperparameters())
loss_plot('loss_values.csv')
plot_mee('mee_values.csv')
test_model_temp(x_test, y_test, model[0], model[1], network.get_layers(), network.get_hyperparameters()['metric'])

In [ ]:
network = Net('./config_cup.yaml')
network.print_structure()

results = train_and_evaluate(x_train, y_train, x_test, y_test, network.get_W(), network.get_b(), network.get_layers(), network.get_hyperparameters())

for key in results.keys():
    if key != 'model':
        print(key, ' : ', results[key])

In [ ]:
#load data TEST
data_TS = pd.read_csv(r'C:\Users\Tano\Documents\UNI\MAGISTRALE\ML\nn_from_scratch\datasets\cup\ML-CUP24-TS.csv', header=None, skiprows=7)
data_TS = data_TS.to_numpy()
x_TS = data_TS[:,1:]
print(x_TS.shape)

cup_labels = blind_test(x_TS, results['model'][0], results['model'][1], network.get_layers())
#save cup_labels in a csv file
np.savetxt("cup_labels.csv", cup_labels, delimiter=",")
# print the first 10 labels
print(cup_labels[:10])

In [ ]:
# plot the dataset and the cup_labels in 3d
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(cup_labels[:,0], cup_labels[:,1], cup_labels[:,2], c='r', marker='o')
ax.scatter(y_train[:,0], y_train[:,1], y_train[:,2], c='b', marker='o')
plt.show()